<a href="https://colab.research.google.com/github/Starrysmh/MARK5828-AdvertisingAnalytics-Project/blob/main/Copy_of_6_Feature_Cleaning_and_Collection_of_Features_revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What you need for running this notebook!!

1. Extracted Data (csv files) from running the colab 5. Processing_Images_Using_Google_CV
2. Output from running the colab corresponding to OpenCV
3. Instagram Data (csv/excel sheets) downloaded from phantombuster


# Importing the Libraries

In [1]:
import pandas as pd 
import numpy as np
import os
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
!pip install webcolors
import webcolors
from math import sqrt as sq
from collections import Counter
from string import punctuation
from google.colab import files

Setting the Source path for the extracted google vision features

In [2]:
src_path='./drive/MyDrive/Advertising_Analytics_T2_2021/B2C_B2B/gsus4/extracted_vision_data/'

Adding Messages to print when we catch an exception

In [3]:
file_not_found_error_msg='Looks like there is no file with the specified name in the specified path!\nPlease check!!'
name_eror_msg='Looks like the variable you are trying to give as input to the function does not exist or a function itself is not defined! \nKindly Check!'

In [4]:
def file_reader(): 
  uploaded_file=files.upload()
  uploaded_file=list(uploaded_file.keys())[0]
  if uploaded_file.endswith('csv'):
    data=pd.read_csv(uploaded_file)
  else:
    data=pd.read_excel(uploaded_file)
  return data

In [5]:
print('Please Upload the data with features extracted using OpenCv:')
opencv_features=file_reader()

Please Upload the data with features extracted using OpenCv:


Saving gsus4_visual_features_1427photo_fromOpenCV.csv to gsus4_visual_features_1427photo_fromOpenCV.csv


In [6]:
print('please upload the instagram data')
insta_data=file_reader()
insta_data['file']=insta_data.index+2

please upload the instagram data


Saving gsus4coinstacrape.csv to gsus4coinstacrape.csv


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Reading the relevant data based on you research question

While this section contains code to pre-process the data from all the 8 APIs. Please use the sections only that are relevant to you!

This section contains the commands necessary to read the data (csv files) with features from google vision which we extracted using the notebook 5.

1. Face Detection 

In [8]:
try:
  face_data=pd.read_csv(src_path+'face_data.csv')
except FileNotFoundError:
  try:
    face_data=pd.read_excel(src_path+'face_data.xlsx')
  except FileNotFoundError:
    print(file_not_found_error_msg)


Function to pre process the features extracted from the Face Detection API.

In [9]:
def pre_proc_face_data(df):
  emotional_ones=['LIKELY','POSSIBLE','VERY_LIKELY',3,4,5]
  df['joy_faces']=df.joy.apply(lambda x:1 if x in emotional_ones else 0)
  df['sorrow_faces']=df.sorrow.apply(lambda x:1 if x in emotional_ones else 0)
  df['surprise_faces']=df.surprise.apply(lambda x:1 if x in emotional_ones else 0)
  df['anger_faces']=df.anger.apply(lambda x:1 if x in emotional_ones else 0)
  df['blurred_binary']=df.blurred.apply(lambda x:1 if x in emotional_ones else 0)
  df=df.groupby('file').sum()
  df.drop('detection_confidence',axis=1,inplace=True)
  df.reset_index(inplace=True)
  df['blurred']=df['blurred_binary'].apply(lambda x:1 if x>=1 else 0)
  df.drop('blurred_binary',axis=1,inplace=True)
  return df

In [10]:
try:
  face_mod=pre_proc_face_data(face_data)
  print(face_mod.head())
except NameError:
  print(name_eror_msg)

          file  anger  sorrow  ...  sorrow_faces  surprise_faces  anger_faces
0  1052_pretty      1       1  ...             0               0            0
1  1053_pretty      1       1  ...             0               0            0
2  1075_pretty      1       1  ...             0               0            0
3  1076_pretty      1       1  ...             0               0            0
4  1100_pretty      1       1  ...             0               0            0

[5 rows x 12 columns]


2. Image Properties Detection

In [11]:
try:
  image_properties_data=pd.read_csv(src_path+'image_properties_data.csv')
except FileNotFoundError:
  try:
    image_properties_data=pd.read_excel(src_path+'image_properties_data.xlsx')
  except FileNotFoundError:
    print(file_not_found_error_msg)

In [12]:
image_properties_data.head()

,file,color,pixel_fraction,color_score
0,1035_pretty,"(13.0, 19.0, 23.0)",0.360000,0.386262
1,1035_pretty,"(243.0, 192.0, 172.0)",0.036200,0.075618
2,1035_pretty,"(21.0, 52.0, 75.0)",0.023933,0.040213
3,1035_pretty,"(242.0, 219.0, 191.0)",0.017067,0.033306
4,1035_pretty,"(33.0, 135.0, 114.0)",0.010867,0.030500


Functions to simply the colors and map them

In [13]:
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

def get_colour_name(requested_colour):
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return closest_name

def match_base_color(requested_color):
  base_colors=['black','white','blue','green','red','orange','yellow','pink']
  
  base_color_dict={c:[webcolors.name_to_rgb(c).red,webcolors.name_to_rgb(c).green,webcolors.name_to_rgb(c).blue] for c in base_colors}
  euc_dist={}
  for k,v in base_color_dict.items():
    euc_dist[(sq((v[0]-requested_color[0])**2+(v[1]-requested_color[1])**2+(v[2]-requested_color[2])**2))]=k
  return euc_dist[min(euc_dist.keys())]

def pre_proc_image_prop(df):
  df1=df.copy()
  df1['color']=df1.color.apply(lambda x:(int(float(x.strip(')').strip('(').split(',')[0])),int(float(x.strip(')').strip('(').split(',')[1])),int(float(x.strip(')').strip('(').split(',')[2]))))

  df1['color_name']=df1.color.apply(lambda x:get_colour_name(x))

  df1['color_name']=df1.color_name.apply(lambda x:match_base_color([webcolors.name_to_rgb(x).red,webcolors.name_to_rgb(x).green,webcolors.name_to_rgb(x).blue]))
  df1.drop(['color','color_score'],axis=1,inplace=True)
  df1.index=df1.file
  df1.drop('file',axis=1,inplace=True)
  df1=pd.get_dummies(df1,columns=['color_name']).mul(df1.pixel_fraction,axis=0)
  df1.drop('pixel_fraction',inplace=True,axis=1)
  df1.reset_index(inplace=True)
  df1=df1.groupby('file').sum()
  df1.reset_index(inplace=True)
  return df1


In [14]:
try:
  image_properties_data_mod=pre_proc_image_prop(df=image_properties_data)
  print(image_properties_data_mod.head())
except NameError:
  print(name_eror_msg)

          file  color_name_black  ...  color_name_white  color_name_yellow
0  1000_pretty          0.033867  ...               0.0                0.0
1  1001_pretty          0.018844  ...               0.0                0.0
2  1002_pretty          0.124311  ...               0.0                0.0
3  1003_pretty          0.025378  ...               0.0                0.0
4  1004_pretty          0.282267  ...               0.0                0.0

[5 rows x 9 columns]


3. Landmark Detection

In [15]:
try:
  landmark_data=pd.read_csv(src_path+'landmark_data.csv')
except FileNotFoundError:
  try:
    landmark_data=pd.read_excel(src_path+'landmark_data.xlsx')
  except:
    print(file_not_found_error_msg)

In [16]:
try:
  landmark_data_mod=pd.get_dummies(landmark_data,columns=['landmark_description'],drop_first=True)
  landmark_data_mod.drop('landmark_score',axis=1,inplace=True)
  landmark_data_mod=landmark_data_mod.groupby('file').sum() #for handling multiple landmarks
  landmark_data_mod.reset_index(inplace=True)
  print(landmark_data_mod.head())
except NameError:
  print(name_eror_msg)

          file  ...  landmark_description_Twain's Tavern
0  1006_pretty  ...                                    0
1  1158_pretty  ...                                    0
2  1204_pretty  ...                                    0
3  1262_pretty  ...                                    0
4  1327_pretty  ...                                    0

[5 rows x 13 columns]


4. Logo Detection

In [17]:
try:
  logo_data=pd.read_csv(src_path+'logo_data.csv')
except FileNotFoundError:
  try:
    logo_data=pd.read_excel(src_path+'logo_data.xlsx')
  except FileNotFoundError:
    print(file_not_found_error_msg)

Helper Functions for Pre Processing

We use two functions to help us: freq_desc and dummify_df.

Freq_Desc takes in a dataframe, column (categorical) and a threshold frequency. 

It retains only those values of the column that appear at least as much as the specified threshold_freq.

These values are then used by the dummify_df and dummy variables are created.



Please note that you may have to change the threshold_freq based on your data and observations.

In [18]:
def freq_desc(df,col_name,threshold_freq=10):
  df_mod=df.groupby(['file',col_name]).max()
  df_mod.reset_index(inplace=True)
  freq_descriptions=df_mod[col_name].value_counts()
  freq_descriptions=list(freq_descriptions[freq_descriptions>=threshold_freq].index.values)
  return freq_descriptions

def dummify_df(df,col_name,threshold_freq=10):
  f=freq_desc(df=df,col_name=col_name,threshold_freq=threshold_freq)
  df_mod=df[df[col_name].isin(f)]
  df_mod=pd.get_dummies(df_mod,columns=[col_name])
  return df_mod


In [19]:
try:
  logo_data_mod=dummify_df(logo_data,'logo_description',threshold_freq=1)
  logo_data_mod=logo_data_mod.groupby('file').sum()
  logo_data_mod.reset_index(inplace=True)
  logo_data_mod.drop('logo_score',axis=1,inplace=True)
  print(logo_data_mod.head())
except NameError:
  print(name_eror_msg)

          file  ...  logo_description_zSpace
0  1006_pretty  ...                        0
1  1008_pretty  ...                        0
2  1012_pretty  ...                        0
3  1014_pretty  ...                        0
4  1015_pretty  ...                        0

[5 rows x 443 columns]


5. Multiple Objects Detection

In [20]:
try:
  multiple_objects_data=pd.read_csv(src_path+'multiple_objects_data.csv')
except FileNotFoundError:
  try:
    multiple_objects_data=pd.read_excel(src_path+'multiple_objects_data.xlsx')
  except FileNotFoundError:
    print(file_not_found_error_msg)

In [21]:
try:
  multiple_objects_data_mod=dummify_df(multiple_objects_data,'object_name',threshold_freq=10)
  multiple_objects_data_mod=multiple_objects_data_mod.groupby('file').sum()
  multiple_objects_data_mod.reset_index(inplace=True)
  multiple_objects_data_mod.drop('object_score',axis=1,inplace=True)
  print(multiple_objects_data_mod.head())
except NameError:
  print(name_eror_msg)

          file  object_name_Bag  ...  object_name_Table top  object_name_Top
0  1000_pretty                0  ...                      0                0
1  1001_pretty                0  ...                      0                0
2  1002_pretty                0  ...                      0                0
3  1003_pretty                0  ...                      0                0
4  1004_pretty                0  ...                      0                0

[5 rows x 36 columns]


6. OCR Detection

In [22]:
try:
  #ocr_data=pd.read_csv(src_path+'ocr_data.csv')
  ocr_data=pd.read_excel(src_path+'ocr_data.xlsx')
except FileNotFoundError:
  print(file_not_found_error_msg)

Function to Preprocess the OCR Text - It helps creates frequency of each word in the text_description.

We have added an additional constraint that a desirable word should be atleast 3 characters long (min_len) and atleast have appeared 30 times (min_freq) in all of the data. 

You may change the min_len and min_freq based on your data and observations.

In [23]:
def preprocess_ocr_text(df,col_name,min_len,min_freq):
  translator = str.maketrans('', '', punctuation)
  df['clean_'+col_name]=df[col_name].apply(lambda x:x.replace('\n',' '))
  df['clean_'+col_name]=df['clean_'+col_name].apply(lambda x:x.upper())
  df['clean_'+col_name]=df['clean_'+col_name].apply(lambda x:x.translate(translator))
  cv_fit=cv.fit_transform(df['clean_'+col_name])
  cv_op=pd.DataFrame(cv_fit.todense(),columns=cv.get_feature_names())
  cols_to_drop=[i for i in cv_op.columns if len(i) < min_len and i!='' and i!=' ']
  cols_to_drop.extend(cv_op.sum()[cv_op.sum()<min_freq].index.values)
  cv_op.drop(cols_to_drop,axis=1,inplace=True)
  cv_op.index=df.file
  cv_op.reset_index(inplace=True)
  return cv_op

In [24]:
try:
  ocr_data_mod=preprocess_ocr_text(df=ocr_data,col_name='ocr_text_description',min_len=3,min_freq=30)
  ocr_data_mod.sort_values(by='file',inplace=True)
  print(ocr_data_mod.head())
except NameError:
  print(name_eror_msg)

             file  100  24ghz  500ma  ...  wwwgsus4comau  xlr  you  your
1100  1000_pretty    0      0      0  ...              0    0    0     0
1101  1001_pretty    0      0      0  ...              0    0    0     0
1102  1002_pretty    0      0      0  ...              0    0    0     0
1103  1003_pretty    0      0      0  ...              0    0    0     0
1104  1004_pretty    0      0      0  ...              0    0    0     0

[5 rows x 199 columns]


7. Safe Search Detection

In [25]:
try:
  safe_search_data=pd.read_csv(src_path+'safe_search_data.csv')
except FileNotFoundError:
  try:
    safe_search_data=pd.read_excel(src_path+'safe_search_data.xlsx')
  except FileNotFoundError:
    print(file_not_found_error_msg)

In [26]:
def pre_proc_safe_search_data(df):
  emotional_ones=['LIKELY','POSSIBLE','VERY_LIKELY',3,4,5]
  df['adult']=df.adult.apply(lambda x:1 if x in emotional_ones else 0)
  df['racy']=df.racy.apply(lambda x:1 if x in emotional_ones else 0)
  df['medical']=df.medical.apply(lambda x:1 if x in emotional_ones else 0)
  df['spoof']=df.spoof.apply(lambda x:1 if x in emotional_ones else 0)
  df['violence']=df.violence.apply(lambda x:1 if x in emotional_ones else 0)
  return df

In [27]:
try:
  safe_search_mod=pre_proc_safe_search_data(safe_search_data)
  print(safe_search_mod.head())
except NameError:
  print(name_eror_msg)

          file  adult  medical  racy  spoof  violence
0  1035_pretty      0        0     0      0         0
1  1036_pretty      0        0     0      0         0
2  1037_pretty      0        0     0      0         0
3  1038_pretty      0        0     0      0         0
4  1042_pretty      0        0     0      0         0


8. Web Entities Detection

In [28]:
try:
  web_data=pd.read_csv(src_path+'web_search_data.csv')
except FileNotFoundError:
  try:
    web_data=pd.read_excel(src_path+'web_search_data.xlsx')
  except FileNotFoundError:
    print(file_not_found_error_msg)

In [29]:
try:
  web_data_mod=dummify_df(df=web_data[web_data.web_score>=0.5],col_name='web_description',threshold_freq=3)
  web_data_mod.drop(['web_score'],inplace=True,axis=1)
  web_data_mod.drop_duplicates(inplace=True)
  web_data_mod=web_data_mod.groupby(['file','best_guess_labels']).sum()
  web_data_mod.reset_index(inplace=True)
  web_data_mod=dummify_df(df=web_data_mod,col_name='best_guess_labels',threshold_freq=1)
  print(web_data_mod.head())
except NameError:
  print(name_eror_msg)

          file  ...  best_guess_labels_캠퍼 플로어
0  1000_pretty  ...                         0
1  1001_pretty  ...                         0
2  1002_pretty  ...                         0
3  1004_pretty  ...                         0
4  1006_pretty  ...                         0

[5 rows x 519 columns]


### 9. Label Detection

In [30]:
try:
  label_data=pd.read_csv(src_path+'label_data.csv')
except FileNotFoundError:
  try:
    label_data=pd.read_excel(src_path+'label_data.xlsx')
  except FileNotFoundError:
    print(file_not_found_error_msg)

In [31]:
try:
  label_data_mod=dummify_df(df=label_data[label_data.label_score>=0.5],col_name='label_description',threshold_freq=0.05*len(label_data.file.unique()))
  label_data_mod.drop(['label_score','topicality'],inplace=True,axis=1)
  label_data_mod=label_data_mod.groupby(['file']).sum()
  label_data_mod.drop_duplicates(inplace=True)
  label_data_mod.reset_index(inplace=True)
  print(label_data_mod.head())
except NameError:
  print(name_error_msg)

          file  ...  label_description_Wood
0  1000_pretty  ...                       0
1  1001_pretty  ...                       0
2  1002_pretty  ...                       0
3  1003_pretty  ...                       0
4  1004_pretty  ...                       0

[5 rows x 45 columns]


Merging the DataFrames


In [32]:
def data_merger(*df):
  n_data_frames=len(df)
  df_merged=df[0]
  for i in range(1,n_data_frames):
    df_merged=df_merged.merge(right=df[i],how='left')
  df_merged.file=df_merged.file.apply(lambda x:x.strip('_pretty'))
  df_merged.file=df_merged.file.astype('int64')
  df_merged.sort_values('file',inplace=True)
  return df_merged

Below is just an example! Please pass the dataframes (related to pre-processed google vision features) dependent on your research question as the input to the function below.

In [33]:
try:
  r=data_merger(logo_data_mod,ocr_data_mod,face_mod,web_data_mod,label_data_mod)
except NameError:
  print(name_eror_msg)

In [34]:
r.shape

(468, 1214)

We have chosen likeCount from the Instagram Data as the target variable. Kindly change the likeCount to commentCount in the code below if your target variable is different!

In [35]:
#Combing OpenCV Data and Google CV Data and instagram data
#reg_data=r.merge(right=opencv_features,left_on='file',right_on='short_code')
reg_data=opencv_features.merge(right=r,left_on='short_code',right_on='file',how='left')
#reg_data.drop('short_code',inplace=True,axis=1)
reg_data.drop('file',inplace=True,axis=1)
reg_data.rename({'short_code':'file'},axis=1,inplace=True)
reg_data.fillna(value=0,axis=0,inplace=True)
reg_data=reg_data.merge(right=insta_data[['file','likeCount']],left_on='file',right_on='file',how='left')
#reg_data.drop('index',inplace=True,axis=1)

Save and Download the File!

In [36]:
reg_data.to_csv('regression_data.csv',index=None)
files.download('regression_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>